# Create Prompt with Plugins Descriptions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/edreisMD/plugnplai/blob/main/examples/create_prompt_plugins.ipynb)

The goal of this example is to load plugins specifications
and create a prefix prompt describing the API to the LLM

## Install

In [1]:
pip install plugnplai -q

Note: you may need to restart the kernel to use updated packages.


## Get the plugins from the directory

We want to install three plugins at maximum to fit the description on the context length

Lets find one plugin for each category:
1. travel
2. shopping 
3. weather

We can use PlugnPlai categories (see [API reference](https://plugnplai.github.io/))

In [2]:
import plugnplai as pl

# Get working plugins - only tested plugins (in progress)
urlsTravel = pl.get_plugins(category='travel')
print(f'Travel plugins: {urlsTravel}')

urlsShopping = pl.get_plugins(category='shopping')
print(f'Shopping plugins: {urlsShopping}')

urlsWeather = pl.get_plugins(category='weather')
print(f'Weather plugins: {urlsWeather}')

Travel plugins: ['https://gogaffl.com', 'https://trip.com', 'https://api.yelp.com', 'https://gps-telecom.com']
Shopping plugins: ['https://pricerunner.com', 'https://server.shop.app', 'https://klarna.com']
Weather plugins: ['https://api.tomorrow.io']


In [3]:
# Lets pick one of each list and add to our url list

urls = []

# Trip (list index 1)
urls.append([plugin for plugin in urlsTravel if 'trip' in plugin][0])

# Klarna (list index 2)
urls.append([plugin for plugin in urlsShopping if 'klarna' in plugin][0])

# Speak (list index 0)
urls.append([plugin for plugin in urlsWeather if 'tomorrow' in plugin][0])

print(f'Our chosen Plugins: {urls}')

Our chosen Plugins: ['https://trip.com', 'https://klarna.com', 'https://api.tomorrow.io']


## Load and activate the plugins

In [4]:
from plugnplai import Plugins

plugins = Plugins.install_and_activate(urls)

## Print the default prompt for the active plugins

In [5]:
print(plugins.prompt)


# SYSTEM MESSAGE
You are a large language model trained to assist humans.
Knowledge Cutoff: 2021-09
Current date: 2023-05-12
Below is a list of available APIs that you can utilize to fulfill user requests. 
When using an API, please follow the specified format to make the API call. 
If possible, avoid asking follow-up questions and aim to complete the task with the information provided by the user.

To make an API call, use the following format:

[API]namespace.operationId[/API]
[PARAMS]{ 
    "parameter_name": "parameter_value",
    ...
}[/PARAMS]

For example, to call an API operation with the operation ID "productsUsingGET" in the "KlarnaProducts" namespace, 
and provide the required parameters "q" and "size", the format would be as follows:

[API]KlarnaProducts.productsUsingGET[/API]
[PARAMS]{
    "q": "t-shirt", 
    "size": 3
}[/PARAMS]

Please ensure that you use the correct namespace and operation ID, and provide the necessary parameters for each API call. 
After requesting th

## Lets look at the length of the prompt

Get the number of tokens of the prompt by just calling 'plugins.tokens' 

In [6]:
print(plugins.tokens)

731


## Customize the prompt template

You can customize the template by just passing 'template' as a variable

Lets adapt this awesome and funny template from this [LangChain example](https://python.langchain.com/en/latest/use_cases/agents/custom_agent_with_plugin_retrieval.html)

The template must have a {{plugins}} variable, to be filled with the descriptions. 

In [7]:
custom_template = '''
Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following APIs:

{{plugins}}
To call any api write the name, operation Id and parameters in the following format:
[API]namespace.operationId[/API]
[PARAMS]{ "parameter_name": "parameter_value", ...}[/PARAMS]

Like in this example to shop a t-shirt: 
[API]KlarnaProducts.productsUsingGET[/API]
[PARAMS]{"q": "t-shirt", "size": 3}[/PARAMS]

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question:
'''

In [8]:
plugins = Plugins.install_and_activate(urls, template = custom_template)
print(plugins.prompt)
print(f'Number of tokens: {plugins.tokens}')


Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following APIs:

### Plugin 1
// Plugin for users to effortlessly get customized travel product recommendation and itinerary planning including hotels and flights using chatGPT.
namespace Trip {

operationId search_flight_ticket = (_: {'originCityCode'*: 'str', 'destinationCityCode'*: 'str', 'departureDate'*: 'str', 'returnDate'*: 'str', 'locale'*: 'str', 'oneWayOrRoundTrip'*: 'str'}) => any

operationId search_hotel = (_: {'cityName'*: 'str', 'topHotel'*: 'int', 'locale'*: 'str', 'checkIn'*: 'any', 'checkOut'*: 'any'}) => any}

### Plugin 2
// Assistant uses the Klarna plugin to get relevant product suggestions for any shopping or product discovery purpose. Assistant will reply with the following 3 paragraphs 1) Search Results 2) Product Comparison of the Search Results 3) Followup Questions. The first paragraph contains a list of the products with their attributes listed clea